
# 📊 **Análisis de Sentimiento Audios**
**Autor:** Enzo Favian Infantes Zuniga \
**Rol:** Data Scientist | Economist \
**Correo:** enzo.infantes28@gmail.com \
**Fecha:** Noviembre 2025  


In [1]:
import os
import pandas as pd
from tqdm import tqdm 

from pydub import AudioSegment

import noisereduce as nr
import librosa
import soundfile as sf

import torch
import torchaudio

C:\Users\einfantesz\AppData\Roaming\Python\Python311\site-packages\pydub\utils.py:170: RuntimeWarning: Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work
  warn("Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work", RuntimeWarning)


In [2]:
current_dir = os.getcwd()

main_dir = os.path.dirname(current_dir)

data_dir = os.path.join(main_dir, 'data')
audio_train_dir = os.path.join(data_dir, 'TRAIN')
audio_test_dir = os.path.join(data_dir, 'TEST')

audio_processed_dir = os.path.join(data_dir, 'processed')
audio_processed_train_dir = os.path.join(audio_processed_dir, 'train')
audio_processed_test_dir = os.path.join(audio_processed_dir, 'test')

### 1.1 Normalización del audio
`AudioSegment`: Lee el archivo de audio y lo convierte en un objeto AudioSegment que puedes manipular. \
`set_frame_rate(1600)`: Cambia la frecuencia de muestreo a 16 kHz. Menos tamaño y más que necesario para voz humana (< 8kHz). \
`set_channels(1)`: Todo el sonido en una sola pista.

In [ ]:
files = [f for f in os.listdir(audio_train_dir) if f.lower().endswith('.wav')]

for filename in tqdm(files, desc="Procesando audios", unit="archivo"):
    input_path = os.path.join(audio_train_dir, filename)
    output_path = os.path.join(audio_processed_train_dir, filename)

    try:
        print(f"Procesando {filename}...")
        audio = AudioSegment.from_file(input_path)
        audio = audio.set_frame_rate(16000).set_channels(1)
        audio.export(output_path, format="wav")
    except Exception as e:
        print(f"⚠️ Error con {filename}: {e}. Saltando...")

### 1.2 Denoising | reducción de ruido

`Librosa`: Lee el archivo de audio y lo convierte en un array NumPy (audio) y devuelve la frecuencia de muestreo (sr). \
`nr.reduce_noise`: Algoritmo de supresión de ruido. Atenúa las frecuencias correspondientes a ruido de fondo. \
`sf`: Guarda el archivo en formato *.wav*

In [ ]:
files = [f for f in os.listdir(audio_processed_train_dir) if f.lower().endswith('.wav')]

for filename in tqdm(files, desc="Procesando audios", unit="archivo"):
    input_path = os.path.join(audio_processed_train_dir, filename)
    output_path = os.path.join(audio_processed_train_dir, filename)

    try:
        print(f"Procesando {filename}...")
        audio, sr = librosa.load(input_path, sr=16000)
        reduced_noise = nr.reduce_noise(y=audio, sr=sr)
        sf.write(output_path, reduced_noise, sr)
    except Exception as e:
        print(f"⚠️ Error con {filename}: {e}. Saltando...")

### 1.3 Normalización de Volumen

`librosa.util.normalize`: Ajusta la amplitud del audio para tener un rango consistente. Audios con una misma escala.

In [ ]:
files = [f for f in os.listdir(audio_processed_train_dir) if f.lower().endswith('.wav')]

for filename in tqdm(files, desc="Procesando audios", unit="archivo"):
    input_path = os.path.join(audio_processed_train_dir, filename)
    output_path = os.path.join(audio_processed_train_dir, filename)

    try:
        print(f"Procesando {filename}...")
        normalized = librosa.util.normalize(input_path)
        sf.write(output_path, normalized, sr)
    except Exception as e:
        print(f"⚠️ Error con {filename}: {e}. Saltando...")

### 1.4 Detección de voz

Procesa todos los .wav en la carpeta indicada. Detecta voz con Silero VAD. Recorta los segmentos y los guarda en voz_segmentos.

In [ ]:
model, utils = torch.hub.load(repo_or_dir='snakers4/silero-vad', model='silero_vad', force_reload=True)
(get_speech_timestamps, _, read_audio, _, _) = utils

files = [f for f in os.listdir(audio_processed_train_dir) if f.lower().endswith('.wav')]

output_dir = os.path.join(audio_processed_train_dir, "voz_segmentos")
os.makedirs(output_dir, exist_ok=True)

for filename in tqdm(files, desc="Procesando audios", unit="archivo"):
    input_path = os.path.join(audio_processed_train_dir, filename)

    try:
        print(f"Procesando {filename}...")
        wav = read_audio(input_path)
        speech_timestamps = get_speech_timestamps(wav, model)

        audio = AudioSegment.from_wav(input_path)

        for i, ts in enumerate(speech_timestamps):
            start_ms = ts['start'] * 1000 / 16000  # convertir muestras a ms (16kHz)
            end_ms = ts['end'] * 1000 / 16000
            segment = audio[start_ms:end_ms]
            segment.export(os.path.join(output_dir, f"{filename}_voz_{i+1}.wav"), format="wav")

        print(f"✅ {len(speech_timestamps)} segmentos guardados para {filename}")

    except Exception as e:
        print(f"⚠️ Error con {filename}: {e}. Saltando...")